In [24]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

import pandas as pd
import time

In [27]:

#creating a dataframe
variables = {'confectionery_name': str(), 'province': str(), 'phone_number': str(), 'cost': int(), 'work_start': time,
'work_end': time, 'confectionery_address': str(), 'hookah': bool(), 'internet': bool(), 'delivery': bool(), 'smoking': bool(), 'open_space': bool(), 'live_music': bool(), 'parking': bool(),
'pos': bool(), 'food_quality': int(), 'service': int(), 'cost_value': int(), 'environment': int(), 'follower': int(), 'lat': float(), 'lon': float()}
df_confectionery = pd.DataFrame(variables, index=[])
df_confectionery.dtypes

confectionery_name        object
province                  object
phone_number              object
cost                       int64
work_start                object
work_end                  object
confectionery_address     object
hookah                      bool
internet                    bool
delivery                    bool
smoking                     bool
open_space                  bool
live_music                  bool
parking                     bool
pos                         bool
food_quality               int64
service                    int64
cost_value                 int64
environment                int64
follower                   int64
lat                      float64
lon                      float64
dtype: object

In [26]:
def changetime(time : 'str'):
    time = time.strip()
    if time == '24':
        time = '00'

    if ':' in time:
        newtime = datetime.strptime(time.strip(), '%H:%M').time()
    else:
        newtime = datetime.strptime(time.strip(), '%H').time()
    return newtime

In [51]:

def Data_confectionery(URL, df_confectioneries):
    try:
        resp = requests.get(URL)
        html = resp.text
        soup = BeautifulSoup(html, "html.parser")

        confectionery_name = soup.find('div', {'class': "venue-name-box"}).find('h1',{'property':'name'}).text.strip()


        province = ((soup.find('span', {'property': "address"}).text).split('،')[0].strip())[:30]

        phone_number = str(soup.find('span', {'property': "telephone"}).text.strip())

        cost = int(soup.find('div', {'class': "price-class"}).find_all('span')[-1].text)
      
        time_work = ((soup.find('ul', {'class': "infolist"}).find_all('li')[3].text).split()[-1])
        if '/' in time_work:
            time_work = time_work.split('/')
            work_start = changetime(time_work[0].split('-')[0])
            work_end =changetime(time_work[1].split('-')[1])
        else:
            time_work = time_work.split('-')
            work_start = changetime(time_work[0])
            work_end =changetime(time_work[1])

        confectionery_address = soup.find('span', {'property': "address"}).text.strip()

        features = soup.find('div', {'class': "venue-features-box"}).text

        hookah = False
        internet = False
        delivery = False
        smoking = False
        open_space = False
        live_music = False
        parking = False
        pos = False

        if "ارسال رایگان" in features:
            delivery = True
        if "دستگاه کارت خوان" in features:
            pos = True
        if "اینترنت رایگان" in features:
            internet = True
        if "سیگار" in features:
            smoking = True
        if "موسیقی زنده" in features:
            live_music = True
        if "قلیان" in features:
            hookah = True
        if "فضای باز" in features:
            open_space = True
        if "پارکینگ" in features:
            parking = True    
 
        food_quality = int((soup.find('ul', {'class': "rates-list"}) \
        .find_all('li')[0].find('div',{'class': 'rate-it'})).attrs['data-rateit-value'])

        service = int((soup.find('ul', {'class': "rates-list"}) \
        .find_all('li')[1].find('div',{'class': 'rate-it'})).attrs['data-rateit-value'])

        cost_value = int((soup.find('ul', {'class': "rates-list"}) \
        .find_all('li')[2].find('div',{'class': 'rate-it'})).attrs['data-rateit-value'])
        
        enviornment = int((soup.find('ul', {'class': "rates-list"}) \
        .find_all('li')[3].find('div',{'class': 'rate-it'})).attrs['data-rateit-value'])
    
        #followers find
        follower = soup.find('span', {'class': 'follow-count'})
        follower = follower.text.strip().replace('دنبال کننده', '')
        follower = int(follower.strip())


        #lat and lon
        location = soup.find('div', {'class': 'map-container'}).find('a')
        location = location.attrs['href'].strip().replace('http://www.google.com/maps/place/', '')
        location = location.split(',')
        lat = float(location[0])
        lon = float(location[1])


        new_row = {'confectionery_name': confectionery_name, 'province': province, 'phone_number': phone_number, 'cost': cost, 'work_start': work_start,
        'work_end': work_end, 'confectionery_address': confectionery_address, 'hookah': hookah, 'internet': internet, 'delivery': delivery, 'smoking': smoking, 'open_space': open_space, 'live_music': live_music, 'parking': parking,
        'pos': pos, 'food_quality': food_quality, 'service':service, 'cost_value':cost_value, 'environment': enviornment, 'follower': follower,'lat': lat, 'lon': lon}
        df_confectioneries = df_confectioneries.append(new_row, ignore_index = True)
        
        
        
    except:
        pass
    return df_confectioneries

In [28]:
def Confectionery_link(URL, confectioneries_links):
    resp = requests.get(URL)
    html = resp.text
    soup = BeautifulSoup(html, "html.parser")
    
    for item in soup.find_all('div',{'class': 'restaurant-list-items span-4 th-span-6 mn-span-12'}):
        link = item.find('a', {'class': 'restaurant-link'}).attrs['href'] 
        if link not in confectioneries_links:
            confectioneries_links.append(link)

    
    return confectioneries_links

In [29]:
#extract  pages_number of them
URL = "https://fidilio.com/confectioneries/in/tehran/"
resp = requests.get(URL)
html = resp.text
soup = BeautifulSoup(html, "html.parser")


   
last_page = soup \
.find("div", {"id": "container"}) \
.find_all('a')[-1].attrs['href']
print(last_page)
    
URL = last_page
resp = requests.get(URL)
html = resp.text
soup = BeautifulSoup(html, "html.parser")

pages_max = soup \
.find("div", {"id": "container"}) \
.find_all('a')[-3].string

    


print(pages_max)

https://fidilio.com/confectioneries/in/tehran/?p=11
12


In [16]:
#extract all page_links

confectioneries_links = []

for num in range(int(pages_max)):
    URL = f"https://fidilio.com/confectioneries/in/tehran/?p={num}"
    print(URL)
    Confectionery_link(URL,confectioneries_links)
   


https://fidilio.com/confectioneries/in/tehran/?p=0
https://fidilio.com/confectioneries/in/tehran/?p=1
https://fidilio.com/confectioneries/in/tehran/?p=2
https://fidilio.com/confectioneries/in/tehran/?p=3
https://fidilio.com/confectioneries/in/tehran/?p=4
https://fidilio.com/confectioneries/in/tehran/?p=5
https://fidilio.com/confectioneries/in/tehran/?p=6
https://fidilio.com/confectioneries/in/tehran/?p=7
https://fidilio.com/confectioneries/in/tehran/?p=8
https://fidilio.com/confectioneries/in/tehran/?p=9
https://fidilio.com/confectioneries/in/tehran/?p=10
https://fidilio.com/confectioneries/in/tehran/?p=11


In [33]:
# extract data from each convfectionery and add to dataframe
url ='https://fidilio.com'

for  links in confectioneries_links:
    
    URL = f"{url}{links}"
    df_confectionery = Data_confectionery(URL, df_confectionery)

In [22]:
df_confectionery.describe()

,cost,food_quality,service,cost_value,environment,follower,lat,lon
count,197.000000,197.000000,197.000000,197.000000,197.000000,197.0,197.000000,197.000000
mean,1.934010,1.619289,1.497462,1.426396,1.527919,0.0,35.746194,51.415145
std,0.268614,2.083030,1.970767,1.862880,1.970570,0.0,0.039897,0.064598
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,35.620937,51.134706
25%,2.000000,0.000000,0.000000,0.000000,0.000000,0.0,35.718682,51.383210
50%,2.000000,0.000000,0.000000,0.000000,0.000000,0.0,35.744642,51.423188
75%,2.000000,4.000000,4.000000,3.000000,4.000000,0.0,35.777159,51.459767
max,3.000000,5.000000,5.000000,5.000000,5.000000,0.0,35.819832,51.532235


In [34]:
#fix food_quality columns where value is greater than 5
def f(x):
    if x>5:
        return 5
df_confectionery.loc[(df_confectionery['food_quality'] >= 5),'food_quality'] = 5
df_confectionery

,confectionery_name,province,phone_number,cost,work_start,work_end,confectionery_address,hookah,internet,delivery,...,live_music,parking,pos,food_quality,service,cost_value,environment,follower,lat,lon
0,شیرینی فروشی فرانسه,خيابان انقلاب نبش خيابان ابوريحان,,2,09:00:00,21:00:00,خيابان انقلاب نبش خيابان ابوريحان,False,False,False,...,False,False,True,5,4,4,4,0,35.701143,51.400315
1,شیرینی فروشی گاندی,میدان ونک,88795199,2,08:00:00,21:00:00,میدان ونک، ابتدای خیابان گاندی، پاساژ ۱۱۰ ، طب...,False,False,False,...,False,False,True,4,3,3,4,0,35.756393,51.411995
2,کافه اوریانت,خیابان مفتح.نرسیده به انقلاب,88347810,2,08:30:00,20:00:00,خیابان مفتح.نرسیده به انقلاب,False,False,False,...,False,False,True,4,4,4,4,0,35.702395,51.425835
3,,خیابان یوسف آباد,88027072,1,10:00:00,21:00:00,خیابان یوسف آباد ، خیابان فتحی شقاقی ، میدان ...,False,False,False,...,False,False,True,4,4,4,4,0,35.724867,51.402417
4,شیرینی فروشی طلایی,خیابان میرزای شیرازی,88343141,2,08:30:00,21:30:00,خیابان میرزای شیرازی ، بالاتر از پمپ بنزین ، پ...,False,False,False,...,False,False,True,4,4,4,4,0,35.721634,51.415910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,شیرینی فروشی بابا نوئل,خیابان ظفر دستگردی,22228618,2,08:00:00,22:30:00,خیابان ظفر دستگردی، روبروی خیابان نفت,False,False,False,...,False,False,True,0,0,0,0,0,35.766504,51.429411
193,شیرینی فروشی باقلوا تادو,صادقیه,09128117101,2,11:00:00,22:00:00,صادقیه،اول اشرفی اصفهانی،جلال ال احمد، زیر پل ...,False,False,False,...,False,True,True,0,0,0,0,0,35.725473,51.349052
194,"کافه,بوتیک کیک,شیرینی فروشی لوندر (جردن)",جردن,22414672,2,07:30:00,22:00:00,جردن،نبش دستگردی،پلاک۹۹,False,False,True,...,False,False,True,0,0,0,0,0,35.768085,51.420012
195,شیرینی فروشی باقلوا تک سین,بلوار ميرداماد,26401456,2,09:00:00,23:00:00,بلوار ميرداماد،جنب مترو ميرداماد،نبش برج رز,False,False,False,...,False,False,True,0,0,0,0,0,35.760322,51.433902


In [23]:
df_confectionery.to_csv('confectionery_1.csv', index=False)

In [50]:
# save data on mysql
'''
from sqlalchemy import create_engine, Table, insert
import mysql.connector

user = 'user_group3'
password = 'hrsAWxaEJkIX_group3'
host = '45.139.10.138:80'
db = 'group3'

for index, row in df_1.iterrows():
    engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{db}")
    print(engine.table_names())
    query_1="INSERT INTO  group3.confectionery (confectionery_name  ,province ,phone_number, cost, work_start, work_end) VALUES(%s,%s,%s,%s,%s,%s)"
    my_data_1=(row['confectionery_name'],  row['province'],  row['phone_number'],row['cost'], row['work_start'], row['work_end'])
    id=engine.execute(query_1,my_data_1)

    confectionery_id = id.lastrowid
    print("ID of Row Added  = ",id.lastrowid)
    
    query_2="INSERT INTO  group3.confectionery_address (confectionery_id, confectionery_address) VALUES(%s,%s)"
    my_data_2=(confectionery_id, row['confectionery_address'])
    id_2 = engine.execute(query_2,my_data_2)


    query_3="INSERT INTO  group3.confectionery_rating (confectionery_id, food_quality, service_quality, cost, cost_value, environment) VALUES(%s,%s,%s,%s,%s,%s)"
    my_data_3=(confectionery_id, row['food_quality'], row['service'], row['cost'], row['cost_value'], row['environment'])
    id_3 = engine.execute(query_3,my_data_3)


    query_4="INSERT INTO  group3.cafe_features (cafe_id, hookah, internet, delivery, smoking, open_space, live_music, parking, pos) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    my_data_4=(confectionery_id, row['hookah'], row['internet'], row['delivery'], row['smoking'], row['open_space'], row['live_music'], row['parking'], row['pos'])
    id_4 = engine.execute(query_4,my_data_4)
    '''

C:\Users\morte\AppData\Local\Temp\ipykernel_13396\3572196295.py:11: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  print(engine.table_names())


['cafe', 'cafe_address', 'cafe_features', 'cafe_rating', 'confectionery', 'confectionery_address', 'confectionery_features', 'confectionery_rating']
ID of Row Added  =  1
['cafe', 'cafe_address', 'cafe_features', 'cafe_rating', 'confectionery', 'confectionery_address', 'confectionery_features', 'confectionery_rating']
ID of Row Added  =  2
['cafe', 'cafe_address', 'cafe_features', 'cafe_rating', 'confectionery', 'confectionery_address', 'confectionery_features', 'confectionery_rating']
ID of Row Added  =  3
['cafe', 'cafe_address', 'cafe_features', 'cafe_rating', 'confectionery', 'confectionery_address', 'confectionery_features', 'confectionery_rating']
ID of Row Added  =  4
['cafe', 'cafe_address', 'cafe_features', 'cafe_rating', 'confectionery', 'confectionery_address', 'confectionery_features', 'confectionery_rating']
ID of Row Added  =  5
['cafe', 'cafe_address', 'cafe_features', 'cafe_rating', 'confectionery', 'confectionery_address', 'confectionery_features', 'confectionery_ratin